# Data recording


Sometimes it is useful to store the snapshots of data during the calculation for some tasks such as debugging. QURI Parts provides the smart way to do it with `@recordable` decorator.

Here we create a function that calculates the expectation value of given operators with given quantum state and returns the highest value. Using recorder feature you can store all estimates and retrieve the data outside of the function.

## Setup 
As an example for demonstration, we first create 10 random operators and a random quantum state. Then we implement a recordable function whose purpose is to estimate the expectation value of the 10 operators on the random state and return the maximal estimate.

In [ ]:
import random
import numpy as np

from quri_parts.circuit import QuantumCircuit
from quri_parts.core.operator import Operator, PauliLabel
from quri_parts.core.state.state_helper import quantum_state


qubit_count = 5

# create random state
circuit = QuantumCircuit(qubit_count)
for i in range(qubit_count):
    circuit.add_RX_gate(i, random.random() * 2 * np.pi)
    circuit.add_RY_gate(i, random.random() * 2 * np.pi)
state = quantum_state(qubit_count, circuit=circuit)

# create 10 random operators
ops = []
for _ in range(10):
    coef = random.random()
    p_ids = []
    p_indices = []
    for i in range(qubit_count):
        id = random.randint(0, 3)
        if id != 0:
            p_ids.append(id)
            p_indices.append(i)
    ops.append(Operator({PauliLabel.from_index_and_pauli_list(p_indices, p_ids): coef}))

## Implement and run the recordable function

Here, we implement the `return_highest_estimate` function that iterates over a list of random `Operator`s and a quantum state for estimation.

In [ ]:
from quri_parts.core.utils.recording import Recorder, recordable
from quri_parts.qulacs.estimator import create_qulacs_vector_estimator
from quri_parts.core.state import CircuitQuantumState

# This function returns the highest expectation value of the given operators
# Add @recordable decorator to the function to make it recordable
@recordable
def return_highest_estimate(
    recorder: Recorder, ops: list[Operator], state: CircuitQuantumState
) -> float:
    estimator = create_qulacs_vector_estimator()
    est_vals = []
    for i, op in enumerate(ops):
        est_vals.append(estimator(op, state).value.real)
        recorder.info("expectation value {}".format(i), est_vals[i])
        recorder.debug("operator", op)
    return max(est_vals)

Note that a recordable function is supposed to take a `Recorder` as its first argument. However, with the help of the `@recordable` decorator, you can ignore the `recorder` argument when you just want to call it like a regular function. In this case, nothing will be reocorded.

In [ ]:
return_highest_estimate(ops, state)

0.07902770779990684

Calling it like a usual function does not record anything. To activate recording, you would need to create a `RecordingSession`. Afterwards, we need to set the recording level the function. We provide 2 recording levels `INFO` and `DEBUG`.

- When logging level is `INFO`, only data recorded by `.info` in a recordable function will be recorded.
- When logging level is `DEBUG`, data recorded by `.info` and `.debug` in a recordable function will both be recorded.

With this in mind, let's present how to perform recording.

In [ ]:
from quri_parts.core.utils.recording import INFO, RecordSession

# Create a record session and set the record level
session = RecordSession()
session.set_level(INFO, return_highest_estimate)

# Create a record context and run the function in it
with session.start():
    # You don't need to pass the recorder to the function when you call it
    highest = return_highest_estimate(ops, state)
    print(f"Highest estimate: {highest}")

Highest estimate: 0.07902770779990684


Finally, let's retrieve the data from the record session. Each call to a recordable function creates a `RecordGroup`. A `RecordGroup` contains all the values recodered inside a recordable function. In this case, `return_highest_estimate` is only called once in the session. Thus only 1 group is present. The data is retrieved by:

In [ ]:
rec_group = list(session.get_records().get_history(return_highest_estimate))[0]
print(rec_group)

estimates = [entry.data[1] for entry in rec_group.entries]
print("Estimates:")
print(sorted(estimates))

RecordGroup(
  func_id: __main__.return_highest_estimate,
  entries: [
    INFO:__main__.return_highest_estimate:('expectation value 0', -0.7297514249719795),
    INFO:__main__.return_highest_estimate:('expectation value 1', 0.0037824267979559055),
    INFO:__main__.return_highest_estimate:('expectation value 2', -0.03933141019807398),
    INFO:__main__.return_highest_estimate:('expectation value 3', -0.00020803153850909353),
    INFO:__main__.return_highest_estimate:('expectation value 4', 0.07902770779990684),
    INFO:__main__.return_highest_estimate:('expectation value 5', 0.0009748426947714661),
    INFO:__main__.return_highest_estimate:('expectation value 6', -0.02607452054619854),
    INFO:__main__.return_highest_estimate:('expectation value 7', -0.03516053922826925),
    INFO:__main__.return_highest_estimate:('expectation value 8', -0.0004264415846573135),
    INFO:__main__.return_highest_estimate:('expectation value 9', 4.129521325103165e-05),
  ]
)
Estimates:
[-0.729751424971

For completeness, we also show what happens if the record level is set to `DEBUG`.

In [ ]:
from quri_parts.core.utils.recording import DEBUG

# Create a record session and set the record level
debug_session = RecordSession()
debug_session.set_level(DEBUG, return_highest_estimate)

# Create a record context and run the function in it
with debug_session.start():
    # You don't need to pass the recorder to the function when you call it
    highest = return_highest_estimate(ops, state)


print(
    next(debug_session.get_records().get_history(return_highest_estimate))
)

RecordGroup(
  func_id: __main__.return_highest_estimate,
  entries: [
    INFO:__main__.return_highest_estimate:('expectation value 0', -0.7297514249719795),
    DEBUG:__main__.return_highest_estimate:('operator', {PauliLabel({(1, 1), (3, 2)}): 0.977259042565072}),
    INFO:__main__.return_highest_estimate:('expectation value 1', 0.0037824267979559055),
    DEBUG:__main__.return_highest_estimate:('operator', {PauliLabel({(1, 1), (0, 3), (2, 1), (3, 2)}): 0.19113742808799938}),
    INFO:__main__.return_highest_estimate:('expectation value 2', -0.03933141019807398),
    DEBUG:__main__.return_highest_estimate:('operator', {PauliLabel({(1, 2), (2, 1), (4, 2)}): 0.4295134194173428}),
    INFO:__main__.return_highest_estimate:('expectation value 3', -0.00020803153850909353),
    DEBUG:__main__.return_highest_estimate:('operator', {PauliLabel({(0, 3), (1, 3), (3, 3), (4, 3)}): 0.1351681060631349}),
    INFO:__main__.return_highest_estimate:('expectation value 4', 0.07902770779990684),
    DE